<a href="https://colab.research.google.com/github/hei1sme/MAI391_project/blob/main/MAI391_SE194127_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>